# Notebook IV - Soil Constraints

<hr>
This module, after applying climatic constraints, will be continued to apply yield reduction due to soil constraints. Soil constraints works with combination of seven soil qualities into soil ratings (a single yield reduction factor). This module works with soil properties provided from soil map and each soil mapping unit's unique physical and chemical properties, and LUT-specific soil suitability
rating as excel sheets

Prepared by Geoinformatics Center, AIT
<hr>

### Google drive connection
In this step, we will connect to Google Drive service and mount the drive where we will start our PyAEZ project

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

Then, installing any additional python packages that required to run PyAEZ.
If working on your own PC/machine, these additional installation will vary depending on what is already installed in your Python library. 

In [ ]:
# 'Installing neccessary packages'
# !pip install gdal

## Importing Libraries

In [ ]:
'''import supporting libraries'''
# import pyaez
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
try:
    from osgeo import gdal
except:
    import gdal
import sys

Setting the working directory -- where our PyAEZ project is located.

In [ ]:
'Set the working directory'
work_dir = r'D:\test_working_folder'  # Please change this to your working directory
os.chdir(work_dir)
sys.path.append('D:\PyAEZ_iiasa')
os.getcwd()


Check and create data output folder

In [ ]:
import os
folder_path = './data_output/NB4/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")


## Applying Soil Constraints

In [ ]:
'''importing library'''

from pyaez import SoilConstraints
soil_constraints = SoilConstraints.SoilConstraints()

from pyaez import UtilitiesCalc
obj_utilities = UtilitiesCalc.UtilitiesCalc()

In [ ]:
'''reading data'''
basefilepath = r'D:\PyAEZ_iiasa\data_input\LAO_Admin.tif'

yield_map_rain = gdal.Open(r'./data_output/NB3/clim_maiz_yld_rain.tif').ReadAsArray()
yield_map_irr = gdal.Open(r'./data_output/NB3/clim_maiz_yld_irr.tif').ReadAsArray()

soil_map = gdal.Open(r'D:\PyAEZ_iiasa\data_input/Lao_Soil.tif').ReadAsArray()

(New) All Soil Suitability Ratings can now be provided as excel sheet.

In [ ]:
soil_constraints.importSoilReductionSheet(rain_sheet_path=r'D:\PyAEZ_iiasa\data_input\maiz_soil_params_rain.xlsx',
                             irr_sheet_path=r'D:\PyAEZ_iiasa\data_input\maiz_soil_params_irr.xlsx')

In [ ]:
'''calculate soil qualities and ratings and applying soil constraints - Rain-fed'''

soil_constraints.calculateSoilQualities(irr_or_rain='R', topsoil_path= r'D:\PyAEZ_iiasa\data_input\soil_characteristics_topsoil.xlsx',
                                         subsoil_path= r'D:\PyAEZ_iiasa\data_input\soil_characteristics_subsoil.xlsx') # I: Irrigated, R: Rain-fed

# Calculate Soil rating for each SMU at a given input/management level
soil_constraints.calculateSoilRatings('H') # L: Low, I: Intermediate, H: High input

In [ ]:
np.set_printoptions(precision = 2)
# Let's see each soil qualities calculated
print( soil_constraints.getSoilQualities())

# The 2D array of the soil quality: 0-axis (Row) is the total number of SMUs; 1-axis (Column) corresponds to
# seven soil qualities 
print(soil_constraints.getSoilQualities().shape)

From the calculated soil qualities, the next step is to calculated soil rating for edaphic yield reduction.

In [ ]:
soil_constraints.calculateSoilRatings(input_level= 'H')

# 1st column: SMUs, 2nd column, soil constraint factors
print(soil_constraints.getSoilRatings())

In [ ]:
yield_map_rain_m4 = soil_constraints.applySoilConstraints(soil_map, yield_map_rain)

## get classified output
# yield_map_rain_class_m4 = obj_utilities.classifyFinalYield(yield_map_rain_m4)

fc4_rain = soil_constraints.getSoilSuitabilityMap()

In [ ]:
'''visualize results'''
plt.figure(1, figsize=(25,9))
plt.subplot(1,3,1)
plt.imshow(yield_map_rain, vmax = np.max([yield_map_rain_m4, yield_map_rain]))
plt.colorbar(shrink=0.8)
plt.title('Original Rainfed Yield Maize')

plt.subplot(1,3,2)
plt.imshow(yield_map_rain_m4, vmax = np.max([yield_map_rain_m4, yield_map_rain]))
plt.colorbar(shrink=0.8)
plt.title('Soil Constrainted Rainfed Yield Maize')

plt.subplot(1,3,3)
plt.imshow(fc4_rain, vmax = 1, vmin = 0)
plt.colorbar(shrink=0.8)
plt.title('Soil Constraint Factor (fc4) Rainfed')


In [ ]:
# ''save result'''

obj_utilities.saveRaster(basefilepath, './data_output/NB4/soil_clim_yld_maiz_rain.tif', yield_map_rain_m4)
obj_utilities.saveRaster(basefilepath, './data_output/NB4/maiz_fc4_rain.tif', fc4_rain)

Applying soil constraints for irrigated conditions

In [ ]:
'''calculate soil qualities and ratings and applying soil constraints - Irrigated'''

soil_constraints.calculateSoilQualities(irr_or_rain='I', topsoil_path= r'D:\PyAEZ_iiasa\data_input\soil_characteristics_topsoil.xlsx',
                                         subsoil_path= r'D:\PyAEZ_iiasa\data_input\soil_characteristics_subsoil.xlsx') # I: Irrigated, R: Rain-fed

# Calculate Soil rating for each SMU at a given input/management level
soil_constraints.calculateSoilRatings('H') # L: Low, I: Intermediate, H: High input

In [ ]:
np.set_printoptions(precision = 2)
# Let's see each soil qualities calculated
print( soil_constraints.getSoilQualities())

# The 2D array of the soil quality: 0-axis (Row) is the total number of SMUs; 1-axis (Column) corresponds to
# seven soil qualities 
print(soil_constraints.getSoilQualities().shape)

In [ ]:
soil_constraints.calculateSoilRatings(input_level= 'H')

# 1st column: SMUs, 2nd column, soil constraint factors
print(soil_constraints.getSoilRatings())

In [ ]:
yield_map_irr_m4 = soil_constraints.applySoilConstraints(soil_map, yield_map_irr)

## get classified output
# yield_map_rain_class_m4 = obj_utilities.classifyFinalYield(yield_map_rain_m4)

fc4_irr = soil_constraints.getSoilSuitabilityMap()

In [ ]:
'''visualize results'''
plt.figure(1, figsize=(25,9))
plt.subplot(1,3,1)
plt.imshow(yield_map_irr, vmax = np.max([yield_map_irr_m4, yield_map_irr]))
plt.colorbar(shrink=0.8)
plt.title('Original Irrigated Yield Maize')

plt.subplot(1,3,2)
plt.imshow(yield_map_irr_m4, vmax = np.max([yield_map_irr_m4, yield_map_irr]))
plt.colorbar(shrink=0.8)
plt.title('Soil Constrainted Irrigated Yield Maize')

plt.subplot(1,3,3)
plt.imshow(fc4_rain, vmax = 1, vmin = 0)
plt.colorbar(shrink=0.8)
plt.title('Soil Constraint Factor (fc4) Irrigated')

In [ ]:
# saving the outputs
obj_utilities.saveRaster(basefilepath, './data_output/NB4/soil_clim_yld_maiz_rain.tif',yield_map_irr_m4)
obj_utilities.saveRaster(basefilepath, './data_output/NB4/maiz_fc4_irr.tif',fc4_irr)

<hr>

### END OF MODULE 4: SOIL CONSTRAINTS

<hr>